# 📘 Data Merging & Cleaning – Yousra Descriptive Stats Notebook

This notebook loads the original project datasets (sales, weather, kiwo event),  
cleans and standardizes the date format, merges them using a full outer join,  
and performs descriptive statistics, missing value inspection, and prepares the  
data for further feature engineering and modeling.

The goal is to:
1. Combine **all available data** on matching dates  
2. Perform a clear descriptive exploration  
3. Detect and handle missing values  
4. Produce a clean dataset ready for analysis  


In [ ]:
import pandas as pd
import numpy as np

# Load data from your project folder
df_sales = pd.read_csv("data/kiwo.csv")
df_weather = pd.read_csv("data/wetter.csv")
df_umsatz = pd.read_csv("data/umsatzdaten_gekuerzt.csv")

df_sales.head(), df_weather.head(), df_umsatz.head()
#print number of rows for each dataframe
print(f"Sales Data Rows: {len(df_sales)}")
print(f"Weather Data Rows: {len(df_weather)}")
print(f"Umsatz Data Rows: {len(df_umsatz)}")

In [ ]:
# Convert 'Datum' to proper datetime format and drop invalid rows
for df in (df_sales, df_weather, df_umsatz):
    df["Datum"] = pd.to_datetime(df["Datum"], errors="coerce").dt.normalize()
    df.dropna(subset=["Datum"], inplace=True)

    # Drop duplicate date columns if exist
    if "date" in df.columns:
        df.drop(columns=["date"], inplace=True)

df_sales.info(), df_weather.info(), df_umsatz.info()


In [ ]:
# Adding suffixes helps avoid confusion during merging

df_sales = df_sales.add_suffix("_sales")
df_sales.rename(columns={"Datum_sales": "Datum"}, inplace=True)

df_weather = df_weather.add_suffix("_weather")
df_weather.rename(columns={"Datum_weather": "Datum"}, inplace=True)

df_umsatz = df_umsatz.add_suffix("_umsatz")
df_umsatz.rename(columns={"Datum_umsatz": "Datum"}, inplace=True)


In [ ]:
merged_df = (
    df_sales
    .merge(df_weather, on="Datum", how="outer")
    .merge(df_umsatz, on="Datum", how="outer")
)

merged_df.shape
merged_df.head()


In [ ]:
# Numeric summary
merged_df.describe()

# For all columns (including categorical)
merged_df.describe(include="all")


In [ ]:
missing_count = merged_df.isna().sum()
missing_percent = (merged_df.isna().sum() / len(merged_df)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent
})

missing_df


### ✔️ Missing Value Strategy

**Umsatz (Sales):**
- Missing values mean the day has **no sales** or is **outside the bakery event period**.
- These rows should be **removed**, not imputed.

**Weather Variables:**
- Weather data may be missing because not all dates have weather records.
- Fill missing values using interpolation (numerical)  
  and assign -1 for Wettercode to indicate “unknown weather”.

**Kiwo Event Flag:**
- If missing → fill with 0 (day outside event).


In [ ]:
clean_df = merged_df.copy()

# Remove rows where sales are missing
clean_df = clean_df.dropna(subset=["Umsatz_umsatz"])

# Fill missing Wettercode with category -1
if "Wettercode_weather" in clean_df.columns:
    clean_df["Wettercode_weather"] = clean_df["Wettercode_weather"].fillna(-1)

# Interpolate numerical weather values
for col in clean_df.columns:
    if ("_weather" in col) and (clean_df[col].dtype in ["float64", "int64"]):
        clean_df[col] = clean_df[col].interpolate()

clean_df.isna().sum()


In [ ]:
clean_df["KielerWoche_sales"] = clean_df["KielerWoche_sales"].fillna(0)


In [ ]:
clean_df.isna().sum()


In [ ]:
#drop wettercode_weather column
df_no_weather_code = clean_df.drop(columns=["Wettercode_weather"])
df_no_weather_code.sample(5)


## Adding extra columns

In [ ]:
df_no_weather_code['Datum'] = pd.to_datetime(df_no_weather_code['Datum'], errors='coerce').dt.normalize()
n_invalid = df_no_weather_code['Datum'].isna().sum()
if n_invalid > 0:
    print(f"Warning: {n_invalid} rows have invalid 'Datum' and will have NaT in day columns. Sample:")
    display(df_no_weather_code[df_no_weather_code['Datum'].isna()].head())

# Create integer and name columns for day of week
df_no_weather_code['day_of_week'] = df_no_weather_code['Datum'].dt.weekday  # Monday=0 .. Sunday=6
df_no_weather_code['day'] = df_no_weather_code['Datum'].dt.day_name()

# Optional: make 'day' categorical ordered Monday..Sunday
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_no_weather_code['day'] = pd.Categorical(df_no_weather_code['day'], categories=ordered_days, ordered=True)

print("\nCounts per weekday:")
print(df_no_weather_code['day'].value_counts().sort_index())
df_no_weather_code.sample(5)

### Adding more weather data from meteo archive api

In [ ]:
import requests
from datetime import datetime, date, timedelta
import time
from typing import Union, Iterable, Dict, List
import pandas as pd
import numpy as np

# Simple in-memory cache for API responses
try:
    _OPEN_METEO_CACHE
except NameError:
    _OPEN_METEO_CACHE: Dict[str, Dict[str, float]] = {}

def _iso_date(d: Union[str, date, datetime]) -> str:
    if isinstance(d, str):
        return datetime.fromisoformat(d).date().isoformat()
    if isinstance(d, datetime):
        return d.date().isoformat()
    return d.isoformat()

def fetch_open_meteo_daily_range(
    start_date: Union[str, date, datetime],
    end_date: Union[str, date, datetime],
    latitude: float = 54.3233,
    longitude: float = 10.1228,
    timezone: str = "Europe/Berlin",
    daily_vars: List[str] = None,
    max_retries: int = 3,
    timeout: float = 15.0,
    use_cache: bool = True
) -> pd.DataFrame:
    """
    Fetch daily historical variables from Open-Meteo archive API for a given inclusive date range.
    By default uses Kiel coordinates (lat=54.3233, lon=10.1228).
    daily_vars example: ['precipitation_hours', 'sunshine_duration', 'rain_sum']
    Returns a pandas.DataFrame indexed by date (datetime.date) with columns named after daily_vars.
    """
    if daily_vars is None:
        daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum']

    start_iso = _iso_date(start_date)
    end_iso = _iso_date(end_date)
    start_dt = datetime.fromisoformat(start_iso).date()
    end_dt = datetime.fromisoformat(end_iso).date()
    if end_dt < start_dt:
        raise ValueError("end_date must be >= start_date")

    # Build cache key based on parameters and requested range
    key = f"{latitude:.6f}_{longitude:.6f}_{start_iso}_{end_iso}_{','.join(daily_vars)}_{timezone}"
    if use_cache and key in _OPEN_METEO_CACHE:
        df = _OPEN_METEO_CACHE[key].copy()
        return df

    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_iso,
        "end_date": end_iso,
        "daily": ",".join(daily_vars),
        "timezone": timezone
    }

    last_exc = None
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(base_url, params=params, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            daily = data.get("daily", {})
            times = daily.get("time", []) or []
            if not times:
                raise ValueError("No 'time' in API response daily block.")
            results = {"date": pd.to_datetime(times).date}
            for v in daily_vars:
                vals = daily.get(v, None)
                if vals is None:
                    # If a requested variable is missing, fill with NaN
                    results[v] = [np.nan] * len(times)
                else:
                    # Convert to numeric (floats), keep NaN if parsing fails
                    results[v] = [float(x) if x is not None else np.nan for x in vals]
            df = pd.DataFrame(results, index=pd.to_datetime(times).date)
            df.index.name = "date"
            df = df[[v for v in daily_vars]]  # ensure column order
            if use_cache:
                _OPEN_METEO_CACHE[key] = df.copy()
            return df
        except Exception as e:
            last_exc = e
            if attempt < max_retries:
                time.sleep(0.5 * attempt)
                continue
            raise RuntimeError(f"Failed fetching Open-Meteo archive: {e}") from e


In [ ]:
min_date = df_no_weather_code['Datum'].min()
max_date = df_no_weather_code['Datum'].max()
##min_date = "2013-07-01"
##max_date = "2014-07-30"
print (f"Fetching weather data from {min_date} to {max_date}...")
lat_kiel, lon_kiel = 54.3233, 10.1228
daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum','temperature_2m_mean']
df_extra_weather = fetch_open_meteo_daily_range(min_date, max_date, latitude=lat_kiel, longitude=lon_kiel, daily_vars=daily_vars)
# print columns of df_extended_weather


In [ ]:
df_extra_weather['sunshine_hours'] = df_extra_weather['sunshine_duration'] / 3600.0
#df_extended_weather.head()

# print row count for df_extended_weather and df_no_weather_code
print(f"Extended Weather Data Rows: {len(df_extra_weather)}")
print(f"Sales Data Rows: {len(df_no_weather_code)}")


### Extended Data Frames
Its with extra weather data from open meteo api: df_merged_extended_weather
with rolling 7 day average as well for umsatz

In [109]:
df_extra_weather.index = pd.to_datetime(df_extra_weather.index)
df_merged_extended_weather = df_no_weather_code.merge(
    df_extra_weather,
    left_on='Datum',
    right_index=True,
    how='left'
)

print(f"Merged DataFrame Rows: {len(df_merged_extended_weather)}")
print(f"Merged DataFrame Shape: {df_merged_extended_weather.shape}")
df_merged_extended_weather.head()

Merged DataFrame Rows: 9334
Merged DataFrame Shape: (9334, 15)


,Datum,KielerWoche_sales,Bewoelkung_weather,Temperatur_weather,Windgeschwindigkeit_weather,id_umsatz,Warengruppe_umsatz,Umsatz_umsatz,day_of_week,day,precipitation_hours,sunshine_duration,rain_sum,temperature_2m_mean,sunshine_hours
394,2013-07-01,0.0,6.0,17.8375,15.0,1307011.0,1.0,148.828353,0,Monday,7.0,31025.67,1.0,15.6,8.618242
395,2013-07-01,0.0,6.0,17.8375,15.0,1307012.0,2.0,535.856285,0,Monday,7.0,31025.67,1.0,15.6,8.618242
396,2013-07-01,0.0,6.0,17.8375,15.0,1307013.0,3.0,201.198426,0,Monday,7.0,31025.67,1.0,15.6,8.618242
397,2013-07-01,0.0,6.0,17.8375,15.0,1307014.0,4.0,65.890169,0,Monday,7.0,31025.67,1.0,15.6,8.618242
398,2013-07-01,0.0,6.0,17.8375,15.0,1307015.0,5.0,317.475875,0,Monday,7.0,31025.67,1.0,15.6,8.618242
